### Tailored sourcing - individual assignment

A company buys inventory of four products for its production process. Annual demand, common and product specific ordering cost, unit cost, and holding cost rate are given in the table below. Assume that demand is steady. Find annual operational cost (holding + ordering) for the optimal sourcing strategy.

#### Basic Setup

In [2]:
import pandas as pd

# Provided data
data = {
    "Product": [1, 2, 3, 4],
    "Demand": [1000, 300, 100, 50],
    "Common_Ordering_Cost": [100, 100, 100, 100],  
    "Specific_Ordering_Cost": [10, 20, 25, 25],
    "Unit_Cost": [50, 60, 30, 30],  
    "Holding_Cost_Rate": [0.2, 0.2, 0.2, 0.2]  
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Calculate Holding Cost per unit per year
df['Holding_Cost_Per_Unit'] = df['Unit_Cost'] * df['Holding_Cost_Rate']
df['Fixed_Ordering_Cost'] = df['Common_Ordering_Cost'] + df['Specific_Ordering_Cost']

df

,Product,Demand,Common_Ordering_Cost,Specific_Ordering_Cost,Unit_Cost,Holding_Cost_Rate,Holding_Cost_Per_Unit,Fixed_Ordering_Cost
0,1,1000,100,10,50,0.2,10.0,110
1,2,300,100,20,60,0.2,12.0,120
2,3,100,100,25,30,0.2,6.0,125
3,4,50,100,25,30,0.2,6.0,125


#### 1. Products are sourced independently

In [3]:

# Calculate EOQ for each product
df['EOQ'] = round(((2 * df['Demand'] * df['Fixed_Ordering_Cost']) / df['Holding_Cost_Per_Unit']) ** 0.5, 2)

# Calculate annual ordering cost for each product
df['Annual_Ordering_Cost'] = round((df['Demand'] / df['EOQ']) * df['Fixed_Ordering_Cost'], 2)

# Calculate annual holding cost for each product
df['Annual_Holding_Cost'] = round((df['EOQ'] / 2) * df['Holding_Cost_Per_Unit'], 2)

# Calculate total annual operational cost for each product
df['Total_Annual_Operational_Cost'] = round(df['Annual_Ordering_Cost'] + df['Annual_Holding_Cost'],2)

# Display the updated DataFrame
df[['Product', 'EOQ', 'Annual_Ordering_Cost', 'Annual_Holding_Cost', 'Total_Annual_Operational_Cost']]


,Product,EOQ,Annual_Ordering_Cost,Annual_Holding_Cost,Total_Annual_Operational_Cost
0,1,148.32,741.64,741.60,1483.24
1,2,77.46,464.76,464.76,929.52
2,3,64.55,193.65,193.65,387.30
3,4,45.64,136.94,136.92,273.86


In [4]:
round(df['Total_Annual_Operational_Cost'].sum(),2)

3073.92

#### 2. All four products are sourced with the same frequency

In [5]:
import pandas as pd

# Provided data
data = {
    "Product": [1, 2, 3, 4],
    "Demand": [1000, 300, 100, 50],
    "Common_Ordering_Cost": [100, 100, 100, 100],  
    "Specific_Ordering_Cost": [10, 20, 25, 25],
    "Unit_Cost": [50, 60, 30, 30],  
    "Holding_Cost_Rate": [0.2, 0.2, 0.2, 0.2]  
}

# Convert to DataFrame
df2 = pd.DataFrame(data)

# Calculate Holding Cost per unit per year
df2['Holding_Cost_Per_Unit'] = df2['Unit_Cost'] * df2['Holding_Cost_Rate']

df2

,Product,Demand,Common_Ordering_Cost,Specific_Ordering_Cost,Unit_Cost,Holding_Cost_Rate,Holding_Cost_Per_Unit
0,1,1000,100,10,50,0.2,10.0
1,2,300,100,20,60,0.2,12.0
2,3,100,100,25,30,0.2,6.0
3,4,50,100,25,30,0.2,6.0


In [6]:
# Step1: Calculate the order frequency

S_Asterisk = 100 + df2['Specific_Ordering_Cost'].sum()
numerator = 1000*10 + 300*12 + 100*6 + 50*6
denominator = 2 * S_Asterisk

# Calculate the order frequency for each row using numpy's sqrt for vectorized operations
Order_Frequency = round((numerator / denominator) ** 0.5, 2)

Order_Frequency

6.35

In [7]:
# Step2: Calculate the optimal order size 
df2['Optimal_Order_Size'] = round(df2['Demand'] / Order_Frequency, 2)

df2

,Product,Demand,Common_Ordering_Cost,Specific_Ordering_Cost,Unit_Cost,Holding_Cost_Rate,Holding_Cost_Per_Unit,Optimal_Order_Size
0,1,1000,100,10,50,0.2,10.0,157.48
1,2,300,100,20,60,0.2,12.0,47.24
2,3,100,100,25,30,0.2,6.0,15.75
3,4,50,100,25,30,0.2,6.0,7.87


In [8]:
# Annual order cost
Annual_Order_Cost = Order_Frequency * S_Asterisk
df2['Annual_Holding_Cost'] = df2['Optimal_Order_Size'] * df2['Holding_Cost_Per_Unit'] / 2

Annual_Total_Cost = Annual_Order_Cost + df2['Annual_Holding_Cost'].sum()

Annual_Total_Cost

2284.7

#### 3. Order frequencies are determined according to the tailored aggregation strategy

In [9]:
# Step 1:Identify most frequently ordered product (assume independent ordering) 
df['Order_Frequency'] = round(df['Demand'] / df['EOQ'], 2)
df['Order_Frequency']

0    6.74
1    3.87
2    1.55
3    1.10
Name: Order_Frequency, dtype: float64

In [14]:
# Step2: For all products other than Product 1, recompute frequencies using only product specific ordering costs. (i.e., use s, and set S = 0)
import pandas as pd

# Provided data
data = {
    "Product": [1, 2, 3, 4],
    "Demand": [1000, 300, 100, 50],
    "Common_Ordering_Cost": [100, 0, 0, 0],  
    "Specific_Ordering_Cost": [10, 20, 25, 25],
    "Unit_Cost": [50, 60, 30, 30],  
    "Holding_Cost_Rate": [0.2, 0.2, 0.2, 0.2]  
}

# Convert to DataFrame
df3 = pd.DataFrame(data)

# Calculate Holding Cost per unit per year
df3['Holding_Cost_Per_Unit'] = df3['Unit_Cost'] * df3['Holding_Cost_Rate']
df3['Fixed_Ordering_Cost'] = df3['Common_Ordering_Cost'] + df3['Specific_Ordering_Cost']

# Calculate EOQ for each product
df3['EOQ'] = round(((2 * df3['Demand'] * df3['Fixed_Ordering_Cost']) / df3['Holding_Cost_Per_Unit']) ** 0.5, 2)

df3['Order_Frequency'] = round(df3['Demand'] / df3['EOQ'], 2)

df3


,Product,Demand,Common_Ordering_Cost,Specific_Ordering_Cost,Unit_Cost,Holding_Cost_Rate,Holding_Cost_Per_Unit,Fixed_Ordering_Cost,EOQ,Order_Frequency
0,1,1000,100,10,50,0.2,10.0,110,148.32,6.74
1,2,300,0,20,60,0.2,12.0,20,31.62,9.49
2,3,100,0,25,30,0.2,6.0,25,28.87,3.46
3,4,50,0,25,30,0.2,6.0,25,20.41,2.45


In [17]:
#Step 3: Identify frequency od other products as a multiple mi (assume only product fixed cost) and round up
df3['Order_Frequency_M'] = round(df3['Order_Frequency'].iloc[0] / df3['Order_Frequency'])

df3

,Product,Demand,Common_Ordering_Cost,Specific_Ordering_Cost,Unit_Cost,Holding_Cost_Rate,Holding_Cost_Per_Unit,Fixed_Ordering_Cost,EOQ,Order_Frequency,Order_Frequency_M
0,1,1000,100,10,50,0.2,10.0,110,148.32,6.74,1.0
1,2,300,0,20,60,0.2,12.0,20,31.62,9.49,1.0
2,3,100,0,25,30,0.2,6.0,25,28.87,3.46,2.0
3,4,50,0,25,30,0.2,6.0,25,20.41,2.45,3.0


In [19]:
#Step 4: Recalculate ordering frequency n of most frequently ordered product
# Step 5: Identify ordering frequency of all products, using n and mi

# Product1
numerator2 = 12*300*1 + 6*100*2 + 6*50*3
denominator2 = 2 * (100 + (20/1 + 25/2 + 25/3))

df3['Order_Frequency'].iloc[0] = round((numerator2 / denominator2) ** 0.5, 2)

# Product2
numerator3 = 10*1000*1 + 6*100*2 + 6*50*3
denominator3 = 2 * (100 + (10/1 + 25/2 + 25/3))

df3['Order_Frequency'].iloc[1] = round((numerator3 / denominator3) ** 0.5, 2)

# Product3
numerator4 = 10*1000*1 + 12*300*1 + 6*50*3
denominator4 = 2 * (100 + (10/1 + 20/1 + 25/3))

df3['Order_Frequency'].iloc[2] = round((numerator4 / denominator4) ** 0.5, 2)

# Product4
numerator5 = 10*1000*1 + 12*300*1 + 6*100*2
denominator5 = 2 * (100 + (10/1 + 20/1 + 25/2))

df3['Order_Frequency'].iloc[3] = round((numerator5 / denominator5) ** 0.5, 2)

df3

C:\Users\heyri\AppData\Local\Temp\ipykernel_4564\1801765321.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Order_Frequency'].iloc[0] = round((numerator2 / denominator2) ** 0.5, 2)
C:\Users\heyri\AppData\Local\Temp\ipykernel_4564\1801765321.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Order_Frequency'].iloc[1] = round((numerator3 / denominator3) ** 0.5, 2)
C:\Users\heyri\AppData\Local\Temp\ipykernel_4564\1801765321.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

,Product,Demand,Common_Ordering_Cost,Specific_Ordering_Cost,Unit_Cost,Holding_Cost_Rate,Holding_Cost_Per_Unit,Fixed_Ordering_Cost,EOQ,Order_Frequency,Order_Frequency_M
0,1,1000,100,10,50,0.2,10.0,110,148.32,4.50,1.0
1,2,300,0,20,60,0.2,12.0,20,31.62,6.80,1.0
2,3,100,0,25,30,0.2,6.0,25,28.87,7.24,2.0
3,4,50,0,25,30,0.2,6.0,25,20.41,7.21,3.0


In [20]:
# Calculate the optimal order size 
df3['Optimal_Order_Size'] = round(df2['Demand'] / Order_Frequency, 2)

df3

,Product,Demand,Common_Ordering_Cost,Specific_Ordering_Cost,Unit_Cost,Holding_Cost_Rate,Holding_Cost_Per_Unit,Fixed_Ordering_Cost,EOQ,Order_Frequency,Order_Frequency_M,Optimal_Order_Size
0,1,1000,100,10,50,0.2,10.0,110,148.32,4.50,1.0,157.48
1,2,300,0,20,60,0.2,12.0,20,31.62,6.80,1.0,47.24
2,3,100,0,25,30,0.2,6.0,25,28.87,7.24,2.0,15.75
3,4,50,0,25,30,0.2,6.0,25,20.41,7.21,3.0,7.87


In [21]:
# Change 'Fixed_Ordering_Cost'
df3['Fixed_Ordering_Cost'].iloc[1] = 120
df3['Fixed_Ordering_Cost'].iloc[2] = 125
df3['Fixed_Ordering_Cost'].iloc[3] = 125

df3

C:\Users\heyri\AppData\Local\Temp\ipykernel_4564\732266890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Fixed_Ordering_Cost'].iloc[1] = 120
C:\Users\heyri\AppData\Local\Temp\ipykernel_4564\732266890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Fixed_Ordering_Cost'].iloc[2] = 125
C:\Users\heyri\AppData\Local\Temp\ipykernel_4564\732266890.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Fixed_Ordering_Cost

,Product,Demand,Common_Ordering_Cost,Specific_Ordering_Cost,Unit_Cost,Holding_Cost_Rate,Holding_Cost_Per_Unit,Fixed_Ordering_Cost,EOQ,Order_Frequency,Order_Frequency_M,Optimal_Order_Size
0,1,1000,100,10,50,0.2,10.0,110,148.32,4.50,1.0,157.48
1,2,300,0,20,60,0.2,12.0,120,31.62,6.80,1.0,47.24
2,3,100,0,25,30,0.2,6.0,125,28.87,7.24,2.0,15.75
3,4,50,0,25,30,0.2,6.0,125,20.41,7.21,3.0,7.87


In [23]:
# Annual order cost
df3['Annual_Holding_Cost'] = df3['Optimal_Order_Size'] * df3['Holding_Cost_Per_Unit'] / 2

Annual_Order_Cost = df3['Annual_Holding_Cost'].sum()

Annual_Total_Cost = Annual_Order_Cost + df3['Annual_Holding_Cost'].sum()

round(Annual_Total_Cost, 2)

2283.4